In [40]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests, sys

# Scraping

In [2]:
nasa_news_url="https://mars.nasa.gov/news/"

In [8]:
browser = Browser('chrome', headless=True)
browser.visit(nasa_news_url)

In [9]:
html = browser.html

In [10]:
soup=bs(html,'lxml')

In [11]:
latest_list_item = soup.find("div", class_="list_text")

In [12]:
news_title = latest_list_item.find("div", class_="content_title").text
news_p = latest_list_item.find("div", class_="article_teaser_body").text

print(f"{news_title} : {news_p}")

Robotic Toolkit Added to NASA's Mars 2020 Rover : The bit carousel, which lies at the heart of the rover's Sample Caching System, is now aboard NASA's newest rover. 


# JPL Mars Space Images - Featured Image

In [24]:
## https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars
base_url= "https://www.jpl.nasa.gov"
mars_images_link = f"{base_url}/spaceimages/?search=&category=Mars"

In [25]:
browser = Browser('chrome', headless=True)
browser.visit(mars_images_link)

In [26]:
html = browser.html

In [27]:
soup=bs(html,'lxml')

In [28]:
result = soup.find("footer")

In [29]:
relative_img_url=result.find("a")["data-link"]
print(relative_img_url)

/spaceimages/details.php?id=PIA00063


In [30]:
data_link = f"{base_url}{relative_img_url}"
print (data_link)

https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA00063


In [31]:
#browser = Browser('chrome', headless = True)
browser.visit(data_link)

In [32]:
html = browser.html
soup = bs(html, 'lxml')

In [33]:
hires_image_url=soup.find("article").find("figure", class_="lede").find("a")["href"]
# /spaceimages/images/largesize/PIA17175_hires.jpg


In [34]:
featured_image_url = f"{base_url}{hires_image_url}"
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA00063_hires.jpg


# Mars Weather

In [35]:
twitter_link = "https://twitter.com/marswxreport?lang=en"

In [36]:
response = requests.get(twitter_link)

In [37]:
print(response)

<Response [200]>


In [41]:
soup = bs(response.text, 'lxml')

In [56]:
results = soup.find_all('div', class_="content")

In [73]:
for result in results:
    tweet_text = result.find("p", class_ = "tweet-text")
    tweet_link = result.find("a", class_ = "twitter-timeline-link")
    tweet_date = result.find("span", class_ = "_timestamp")
    if "InSight sol" in result.text:
        print("~" * 100)
        if tweet_link:
            print(f"[{tweet_date.text}] => {tweet_text.text} \n{tweet_link.text}")
            print("Modified String")
            print(tweet_text.text.replace(tweet_link.text,""))
        else:
            print(f"[{tweet_date.text}] => {tweet_text.text}")

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[Aug 15] => InSight sol 253 (2019-08-13) low -100.0ºC (-148.1ºF) high -25.5ºC (-13.9ºF)
winds from the SSE at 4.6 m/s (10.3 mph) gusting to 16.4 m/s (36.6 mph)
pressure at 7.60 hPapic.twitter.com/OnwaHAaYRH 
pic.twitter.com/OnwaHAaYRH
Modified String
InSight sol 253 (2019-08-13) low -100.0ºC (-148.1ºF) high -25.5ºC (-13.9ºF)
winds from the SSE at 4.6 m/s (10.3 mph) gusting to 16.4 m/s (36.6 mph)
pressure at 7.60 hPa
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[Aug 15] => InSight sol 252 (2019-08-12) low -100.8ºC (-149.4ºF) high -26.0ºC (-14.8ºF)
winds from the SSW at 4.4 m/s (9.8 mph) gusting to 18.3 m/s (40.9 mph)
pressure at 7.60 hPapic.twitter.com/WY3JQBXVuU 
pic.twitter.com/WY3JQBXVuU
Modified String
InSight sol 252 (2019-08-12) low -100.8ºC (-149.4ºF) high -26.0ºC (-14.8ºF)
winds from the SSW at 4.4 m/s (9.8 mph) gusting to 1

In [48]:
print(results[1].text)

InSight sol 253 (2019-08-13) low -100.0ºC (-148.1ºF) high -25.5ºC (-13.9ºF)
winds from the SSE at 4.6 m/s (10.3 mph) gusting to 16.4 m/s (36.6 mph)
pressure at 7.60 hPapic.twitter.com/OnwaHAaYRH


In [55]:
for id, result in enumerate(results):
    if "InSight sol" in result.text:
        print("-" * 100)
        print(f"Tweet {id} => {result.text}")

----------------------------------------------------------------------------------------------------
Tweet 1 => InSight sol 253 (2019-08-13) low -100.0ºC (-148.1ºF) high -25.5ºC (-13.9ºF)
winds from the SSE at 4.6 m/s (10.3 mph) gusting to 16.4 m/s (36.6 mph)
pressure at 7.60 hPapic.twitter.com/OnwaHAaYRH
----------------------------------------------------------------------------------------------------
Tweet 2 => InSight sol 252 (2019-08-12) low -100.8ºC (-149.4ºF) high -26.0ºC (-14.8ºF)
winds from the SSW at 4.4 m/s (9.8 mph) gusting to 18.3 m/s (40.9 mph)
pressure at 7.60 hPapic.twitter.com/WY3JQBXVuU
----------------------------------------------------------------------------------------------------
Tweet 4 => InSight sol 251 (2019-08-11) low -101.0ºC (-149.7ºF) high -26.5ºC (-15.8ºF)
winds from the SSE at 4.1 m/s (9.2 mph) gusting to 17.5 m/s (39.1 mph)
pressure at 7.60 hPapic.twitter.com/9mgFzHl8t3
---------------------------------------------------------------------------------